<div style="display: flex; align-items: center; justify-content: center; flex-wrap: wrap;">
    <div style="flex: 1; max-width: 400px; display: flex; justify-content: center;">
        <img src="https://diretorio.bad.pt/wp-content/uploads/2015/09/55c9dc4270c60-IMS-rgb_logo.png" style="max-width: 50%; height: auto; margin-top: 50px; margin-bottom: 50px;margin-left: 6rem;">
    </div>
    <div style="flex: 2; text-align: center; margin-top: 20px;margin-left: 8rem;">
        <div style="font-size: 28px; font-weight: bold; line-height: 1.2;">
            <span style="color:rgb(190, 214, 47);">CIFO Project |</span> <span style="color:rgb(92, 102, 108);">Sports League: Optimization using Genetic Algorithms</span>
        </div>
        <div style="font-size: 17px; font-weight: bold; margin-top: 10px;">
            Spring Semester | 2024 - 2025
        </div>
        <div style="font-size: 17px; font-weight: bold;">
            Master in Data Science and Advanced Analytics
        </div>
        <div style="margin-top: 20px;">
            <div>Diogo Duarte, 20240525</div>
            <div>Rodrigo Sardinha, 20211627</div>
            <div>Rui Luz, 20211628</div>
        </div>
        <div style="margin-top: 20px; font-weight: bold;">
            Group AE
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right,rgb(190, 214, 47), rgb(92, 102, 108));
            padding: 1px; color: white; border-radius: 500px; text-align: center;">
</div>

# Imports

In [1]:
import pandas as pd

# import py files
from Classes import Player, Team, LeagueIndividual
from algorithms.GA_mutation import mutation_swap_players, mutation_regenerate_team, mutation_balance_teams
from algorithms.GA_crossover import team_crossover, position_crossover

# Local Data

In [2]:
df = pd.read_csv("data/players.csv")
df = df.drop(columns=['Unnamed: 0']) # drop the index column

# Problem Configuration

In [3]:
TEAM_SIZE = 7
NUM_TEAMS = 5
BUDGET_LIMIT = 750
TEAM_STRUCTURE = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
POPULATION_SIZE = 20

# Convert DF to player objects

In [4]:
players_by_position = {
    pos: [Player.from_dict(row) for _, row in df[df['Position'] == pos].iterrows()]
    for pos in TEAM_STRUCTURE
}

# Generate Population

In [5]:
# === GENERATE POPULATION ===
def generate_initial_population(size, players_by_position, team_structure, budget_limit, num_teams):
    population = []
    attempts = 0
    max_attempts = 1000 # avoid infinite loop if unable to generate valid leagues

    while len(population) < size and attempts < max_attempts:
        indiv = LeagueIndividual(players_by_position, team_structure, budget_limit, num_teams)
        if indiv.league is not None:
            population.append(indiv)
        attempts += 1

    return population

population = generate_initial_population(
    POPULATION_SIZE,
    players_by_position,
    TEAM_STRUCTURE,
    BUDGET_LIMIT, 
    NUM_TEAMS
)

In [ ]:
# # PRINT ENTIRE POPULATION
# def print_population(population):
#     for i, indiv in enumerate(population):
#         print(f"\n🏟️ League {i+1} | 🧬 Fitness: {indiv.fitness:.4f}")
#         all_names = set()
#         for j, team in enumerate(indiv.league):
#             names = [p.name for p in team.players]
#             positions = [p.position for p in team.players]
#             skill = team.avg_skill()
#             salary = team.total_salary()
#             print(f"  🏆 Team {j+1} | 🧍 Positions: {positions} | 📊 Avg Skill: {skill:.2f} | 💰 Salary: {salary}M")
#
#             # Optional: check for duplicate players within individual
#             duplicates = set(names).intersection(all_names)
#             if duplicates:
#                 print(f"    ⚠️ Duplicate Players: {duplicates}")
#             all_names.update(names)
#
# # Example usage:
# print_population(population)

# Mutations

## #1 Swap Players Betweeen Teams

In [ ]:
swap_players = mutation_swap_players(individual)
print(swap_players)

In [ ]:
if swap_players == individual:
    print("⚠️  No mutation applied (swap failed after multiple attempts).")
else:
    print("✅ Mutation applied successfully.")

### Testing code mutation_swap_players

In [ ]:
# # test mutation_swap_players
# 
# def print_league_details(league_indiv, label=""):
#     print(f"\n{'='*50}\n🏟️  {label} (Fitness: {league_indiv.fitness:.4f})\n{'='*50}")
#     
#     all_players = set()
#     valid = True
# 
#     for i, team in enumerate(league_indiv.league):
#         print(f"\n🏆 Team {i+1}")
#         for player in team.players:
#             print(f"  - {player}")
#         avg = team.avg_skill()
#         total_salary = team.total_salary()
#         print(f"    📊 Avg Skill: {avg:.2f} | 💰 Total Salary: €{total_salary}M")
#         
#         all_players.update(player.name for player in team.players)
# 
#         if not team.is_valid(league_indiv.team_structure, league_indiv.budget_limit):
#             print("    ❌ Invalid team (structure or budget)")
#             valid = False
# 
#     expected_total = sum(league_indiv.team_structure.values()) * len(league_indiv.league)
#     if len(all_players) != expected_total:
#         print("⚠️  Duplicate players found across teams!")
#         valid = False
# 
#     print(f"\n✅ Valid League: {valid}")
#     return valid
# 
# 
# # === GENERATE ONE INDIVIDUAL AND MUTATE IT ===
# original = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, NUM_TEAMS)
# mutated = mutation_swap_players(original)
# 
# # === PRINT RESULTS ===
# print_league_details(original, "Original League")
# print_league_details(mutated, "Mutated League")


## #2 Regenerate a Team Entirely

In [ ]:
regenerate_team, success = mutation_regenerate_team(individual)
print(regenerate_team)

In [ ]:
if regenerate_team == individual:
    print("⚠️  No mutation applied (regenerate team failed).")
else:
    print("✅ Mutation applied successfully.")

### Testing code mutation_regenerate_team

In [ ]:
# def print_player_diff(orig_players, new_players):
#     orig_names = set(p.name for p in orig_players)
#     new_names = set(p.name for p in new_players)
#     added = new_names - orig_names
#     removed = orig_names - new_names
#     return added, removed
#
# def print_detailed_comparison(original, mutated):
#     print(f"\n{'='*60}")
#     print(f"🏟️  Regenerate Team Mutation Comparison")
#     print(f"📈 Fitness: {original.fitness:.4f} → {mutated.fitness:.4f}")
#     print(f"{'='*60}\n")
#
#     # === First Pass: Determine the regenerated team ===
#     max_changes = 0
#     regenerated_team_index = None
#     for i, (team_orig, team_mut) in enumerate(zip(original.league, mutated.league)):
#         orig_names = set(p.name for p in team_orig.players)
#         new_names = set(p.name for p in team_mut.players)
#         changes = len(orig_names.symmetric_difference(new_names))
#         if changes > max_changes:
#             max_changes = changes
#             regenerated_team_index = i
#
#     # === Second Pass: Print teams ===
#     for i, (team_orig, team_mut) in enumerate(zip(original.league, mutated.league)):
#         team_label = f"🏆 Team {i+1}"
#         if i == regenerated_team_index:
#             team_label += " 🆕"
#         print(f"\n{team_label}")
#
#         print(f"Original Team:")
#         for p in team_orig.players:
#             print(f"  - {p}")
#         print(f"    📊 Avg Skill: {team_orig.avg_skill():.2f} | 💰 Salary: €{team_orig.total_salary()}M")
#
#         print(f"Mutated Team:")
#         for p in team_mut.players:
#             print(f"  - {p}")
#         print(f"    📊 Avg Skill: {team_mut.avg_skill():.2f} | 💰 Salary: €{team_mut.total_salary()}M")
#
#         added, removed = print_player_diff(team_orig.players, team_mut.players)
#         if added or removed:
#             print(f"    ➕ Added: {', '.join(added) if added else 'None'}")
#             print(f"    ➖ Removed: {', '.join(removed) if removed else 'None'}")
#
#     print(f"\n✅ League Valid: {all(team.is_valid(mutated.team_structure, mutated.budget_limit) for team in mutated.league)}")
#     print(f"📌 Regenerated Team: Team {regenerated_team_index + 1 if regenerated_team_index is not None else 'Unknown'}")

# # === GENERATE ONE INDIVIDUAL AND MUTATE IT ===
# original = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, NUM_TEAMS)
# mutated, success = mutation_regenerate_team(original)
#
# if success:
#     print_detailed_comparison(original, mutated)
# else:
#     print("⚠️ Mutation failed (could not perform valid regeneration)")

## #3 Balance Teams Average Skill (std dev)

In [ ]:
third_mutation = mutation_balance_teams(individual)
print(third_mutation)

In [ ]:
if third_mutation == individual:
    print("⚠️  No mutation applied (swap failed after multiple attempts).")
else:
    print("✅ Mutation applied successfully.")

### Testing code mutation_balance_teams

In [ ]:
# def print_player_diff(orig_players, new_players):
#     orig_names = set(p.name for p in orig_players)
#     new_names = set(p.name for p in new_players)
#     added = new_names - orig_names
#     removed = orig_names - new_names
#     return added, removed
#
# def print_detailed_comparison(original, mutated):
#     print(f"\n{'='*60}")
#     print(f"🏟️  Mutation Comparison")
#     print(f"📈 Fitness: {original.fitness:.4f} → {mutated.fitness:.4f}")
#     print(f"{'='*60}\n")
#
#     # Identify most-changed team
#     max_changes = 0
#     regenerated_team_index = None
#     for i, (t1, t2) in enumerate(zip(original.league, mutated.league)):
#         orig_names = set(p.name for p in t1.players)
#         new_names = set(p.name for p in t2.players)
#         changes = len(orig_names.symmetric_difference(new_names))
#         if changes > max_changes:
#             max_changes = changes
#             regenerated_team_index = i
#
#     # Print comparison
#     for i, (t1, t2) in enumerate(zip(original.league, mutated.league)):
#         team_label = f"🏆 Team {i+1}" + (" 🆕" if i == regenerated_team_index else "")
#         print(f"\n{team_label}")
#
#         print("Original Team:")
#         for p in t1.players:
#             print(f"  - {p}")
#         print(f"    📊 Avg Skill: {t1.avg_skill():.2f} | 💰 Salary: €{t1.total_salary()}M")
#
#         print("Mutated Team:")
#         for p in t2.players:
#             print(f"  - {p}")
#         print(f"    📊 Avg Skill: {t2.avg_skill():.2f} | 💰 Salary: €{t2.total_salary()}M")
#
#         added, removed = print_player_diff(t1.players, t2.players)
#         if added or removed:
#             print(f"    ➕ Added: {', '.join(added) if added else 'None'}")
#             print(f"    ➖ Removed: {', '.join(removed) if removed else 'None'}")
#
#     print(f"\n✅ League Valid: {all(t.is_valid(mutated.team_structure, mutated.budget_limit) for t in mutated.league)}")
#     print(f"📌 Most changed team: Team {regenerated_team_index + 1 if regenerated_team_index is not None else 'Unknown'}")


In [ ]:
# def test_mutation_balance_teams():
#     from GA_mutation import mutation_balance_teams
#
#     print("\n=== Testing: mutation_balance_teams ===")
#
#     # Generate a sample individual
#     original = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, NUM_TEAMS)
#     mutated = mutation_balance_teams(original)
#
#     # Compare fitness
#     print(f"\n📈 Original Fitness: {original.fitness:.4f}")
#     print(f"📉 Mutated Fitness:  {mutated.fitness:.4f}")
#
#     # Identify changed teams
#     changed_teams = []
#     for i, (t1, t2) in enumerate(zip(original.league, mutated.league)):
#         if set(p.name for p in t1.players) != set(p.name for p in t2.players):
#             changed_teams.append(i + 1)
#
#     if mutated.fitness < original.fitness:
#         print("\n✅ Mutation successful! Fitness improved.")
#     else:
#         print("\n⚠️ No mutation applied. Fitness unchanged.")
#
#     print(f"\n🔁 Changed team(s): {changed_teams if changed_teams else 'None'}")
#
#     # Optionally, print detailed comparison
#     print_detailed_comparison(original, mutated)


In [ ]:
# test_mutation_balance_teams()

# Crossovers

## #1 Crossover by Team

In [ ]:
print(population)

In [ ]:
# Generate Parents
parent1 = population[0]
parent2 = population[1]

In [ ]:
team_co = team_crossover(parent1, parent2)
print(team_co)

### Testing code team_crossover

In [8]:

# # Select parents from existing population
# parent1 = population[0]
# parent2 = population[1]

# # Apply team crossover
# child1, child2 = team_crossover(parent1, parent2)

# # Pretty printer for LeagueIndividuals
# def print_league(individual, label):
#     print(f"\n{label} | 🧬 Fitness: {individual.fitness:.4f}")
#     for i, team in enumerate(individual.league):
#         print(f"🏆 Team {i+1}:")
#         for p in team.players:
#             print(f"    {p.name} ({p.position}) - 📊 Skill: {p.skill} - 💰 Salary: €{p.salary}M")


# # Print parents and children
# print_league(parent1, "👨 Parent 1")
# print_league(parent2, "👩 Parent 2")
# print_league(child1,  "🧒 Child 1 (Team Crossover)")
# print_league(child2,  "🧒 Child 2 (Team Crossover)")


## #2 Crossover by Position

In [ ]:
position_co = position_crossover(parent1, parent2)  
print(position_co)

### Testing code position_crossover

In [10]:
# # === POSITION_CROSSOVER TESTING ===

# # Select parents from existing population
# parent1 = population[0]
# parent2 = population[1]

# # Apply position crossover
# child1_pos, child2_pos = position_crossover(parent1, parent2)

# # Pretty printer for LeagueIndividuals
# def print_league(individual, label):
#     print(f"\n{label} | 🧬 Fitness: {individual.fitness:.4f}")
#     for i, team in enumerate(individual.league):
#         print(f"🏆 Team {i+1}:")
#         for p in team.players:
#             print(f"    {p.name} ({p.position}) - 📊 Skill: {p.skill} - 💰 Salary: €{p.salary}M")

# # Print detailed structure of both parents and children
# print_league(parent1, "👨 Parent 1")
# print_league(parent2, "👩 Parent 2")
# print_league(child1_pos, "👶 Child 1 (Position Crossover)")
# print_league(child2_pos, "👶 Child 2 (Position Crossover)")
